In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
import json

In [3]:
with open('../temp_files/route_dict_journal.json', 'r') as json_file:
    route_dict = json.load(json_file)

In [4]:
route_dict['1']['link_idx_list']

[92, 81, 84, 112]

In [5]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

### create route_choice_probability_matrix $\bf{P} = \left[p_{ir}\right]$ (\# of OD pairs $\times$ \# of routes)

In [6]:
od_nodes_dict = zload('../temp_files/od_nodes_dict_journal.pkz')

In [7]:
od_nodes_dict

{0: {1},
 1: {2, 4, 5},
 2: {3},
 3: {6, 8},
 4: {7, 9},
 5: {10, 11},
 6: {12, 15},
 7: {13, 14},
 8: {16, 17, 18, 19},
 9: {20},
 10: {21},
 11: {22},
 12: {23, 24, 25, 26, 27, 28},
 13: {29},
 14: {30, 31, 32, 33, 34},
 15: {35, 71},
 16: {36},
 17: {37, 38, 39, 40, 41},
 18: {42},
 19: {43, 44, 45, 46, 47},
 20: {48, 74},
 21: {49, 73},
 22: {50, 51},
 23: {52},
 24: {53},
 25: {54, 68, 70},
 26: {55, 56},
 27: {57},
 28: {58},
 29: {59, 60, 72},
 30: {61},
 31: {62, 63, 64, 65, 66},
 32: {67},
 33: {69}}

In [8]:
od_pair_label_dict_full = {}

idx = 0
for origi in range(len(od_nodes_dict)):
    for desti in range(len(od_nodes_dict)):
        if origi != desti:
            od_pair_label_dict_full[idx] = [origi, desti]
            idx += 1
            
zdump(od_pair_label_dict_full, '../temp_files/od_pair_label_dict_full_journal.pkz')

In [9]:
od_list = []
od_list.append(11) # add Boston downtown
for key in od_nodes_dict.keys():
    if len(od_nodes_dict[key]) >= 3:
        od_list.append(key)

In [10]:
od_list

[11, 1, 8, 12, 14, 17, 19, 25, 29, 31]

In [11]:
# od_pair_label_dict = {}

# idx = 0
# for origi in range(len(od_nodes_dict)):
#     for desti in range(len(od_nodes_dict)):
#         if origi != desti:
#             od_pair_label_dict[idx] = [origi, desti]
#             idx += 1
            
# zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [12]:
od_pair_label_dict = {}

idx = 0
for origi in od_list:
    for desti in od_list:
        if origi != desti:
            od_pair_label_dict[idx] = [origi, desti]
            idx += 1
            
zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [13]:
od_pair_label_dict

{0: [11, 1],
 1: [11, 8],
 2: [11, 12],
 3: [11, 14],
 4: [11, 17],
 5: [11, 19],
 6: [11, 25],
 7: [11, 29],
 8: [11, 31],
 9: [1, 11],
 10: [1, 8],
 11: [1, 12],
 12: [1, 14],
 13: [1, 17],
 14: [1, 19],
 15: [1, 25],
 16: [1, 29],
 17: [1, 31],
 18: [8, 11],
 19: [8, 1],
 20: [8, 12],
 21: [8, 14],
 22: [8, 17],
 23: [8, 19],
 24: [8, 25],
 25: [8, 29],
 26: [8, 31],
 27: [12, 11],
 28: [12, 1],
 29: [12, 8],
 30: [12, 14],
 31: [12, 17],
 32: [12, 19],
 33: [12, 25],
 34: [12, 29],
 35: [12, 31],
 36: [14, 11],
 37: [14, 1],
 38: [14, 8],
 39: [14, 12],
 40: [14, 17],
 41: [14, 19],
 42: [14, 25],
 43: [14, 29],
 44: [14, 31],
 45: [17, 11],
 46: [17, 1],
 47: [17, 8],
 48: [17, 12],
 49: [17, 14],
 50: [17, 19],
 51: [17, 25],
 52: [17, 29],
 53: [17, 31],
 54: [19, 11],
 55: [19, 1],
 56: [19, 8],
 57: [19, 12],
 58: [19, 14],
 59: [19, 17],
 60: [19, 25],
 61: [19, 29],
 62: [19, 31],
 63: [25, 11],
 64: [25, 1],
 65: [25, 8],
 66: [25, 12],
 67: [25, 14],
 68: [25, 17],
 69: [2

In [14]:
# number of OD pairs
number_of_od_pairs = len(od_pair_label_dict)

In [15]:
od_pair_label_dict[0]

[11, 1]

In [16]:
od_pair_routes_dict = {}
for row in range(number_of_od_pairs):
    routes = []
    for column in range(len(route_dict)):
        if route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            routes.append(column)
            
    route_length_list = [route_dict[str(column)]['length_of_route'] for column in routes]
    
    refined_routes = []

    number_of_routes_per_OD_pair = 3
    if len(routes) <= number_of_routes_per_OD_pair:
        refined_routes = routes
    else:
        refined_idx_list = [np.argsort(route_length_list)[i] for i in range(number_of_routes_per_OD_pair)]
        refined_routes = [routes[idx] for idx in refined_idx_list]
        
    od_pair_routes_dict[row] = refined_routes

In [17]:
od_pair_routes_dict

{0: [4809, 4810, 4811],
 1: [1909, 1912, 1913],
 2: [2934, 2931, 9083],
 3: [15475, 15476, 15469],
 4: [5418, 5427, 5442],
 5: [5481, 5482, 5483],
 6: [7141, 6052, 7142],
 7: [13511, 13512, 13513],
 8: [12555, 13502, 12556],
 9: [4268, 4269, 4270],
 10: [4271, 4292, 4293],
 11: [1313, 1314, 1315],
 12: [6348, 2877, 6349],
 13: [14757, 14758, 2342],
 14: [10489, 10490, 10491],
 15: [9475, 9476, 9477],
 16: [14511, 98, 14512],
 17: [9318, 9319, 14496],
 18: [7301, 12165, 12166],
 19: [790, 4295, 4296],
 20: [14685, 427, 428],
 21: [3854, 703, 3855],
 22: [4971, 7235, 4950],
 23: [4965, 4966, 15055],
 24: [3399, 1846, 3968],
 25: [2922, 5067, 2923],
 26: [2913, 5106, 2895],
 27: [3752, 3767, 2585],
 28: [2723, 2724, 2725],
 29: [3716, 3698, 3699],
 30: [5803, 10196, 6819],
 31: [14275, 10100, 6903],
 32: [13658, 13659, 15],
 33: [4016, 9860, 15325],
 34: [4704, 7063, 13158],
 35: [4722, 4226, 15334],
 36: [4737, 4738, 3324],
 37: [13221, 2315, 13222],
 38: [832, 833, 11256],
 39: [10043, 

In [18]:
# re-label routes
new_route_label_dict = {}
route_idx = 0
for od_pair_idx in range(len(od_pair_routes_dict)):
    for route in od_pair_routes_dict[od_pair_idx]:
        new_route_label_dict[route_idx] = route
        route_idx += 1

In [19]:
new_route_label_dict

{0: 4809,
 1: 4810,
 2: 4811,
 3: 1909,
 4: 1912,
 5: 1913,
 6: 2934,
 7: 2931,
 8: 9083,
 9: 15475,
 10: 15476,
 11: 15469,
 12: 5418,
 13: 5427,
 14: 5442,
 15: 5481,
 16: 5482,
 17: 5483,
 18: 7141,
 19: 6052,
 20: 7142,
 21: 13511,
 22: 13512,
 23: 13513,
 24: 12555,
 25: 13502,
 26: 12556,
 27: 4268,
 28: 4269,
 29: 4270,
 30: 4271,
 31: 4292,
 32: 4293,
 33: 1313,
 34: 1314,
 35: 1315,
 36: 6348,
 37: 2877,
 38: 6349,
 39: 14757,
 40: 14758,
 41: 2342,
 42: 10489,
 43: 10490,
 44: 10491,
 45: 9475,
 46: 9476,
 47: 9477,
 48: 14511,
 49: 98,
 50: 14512,
 51: 9318,
 52: 9319,
 53: 14496,
 54: 7301,
 55: 12165,
 56: 12166,
 57: 790,
 58: 4295,
 59: 4296,
 60: 14685,
 61: 427,
 62: 428,
 63: 3854,
 64: 703,
 65: 3855,
 66: 4971,
 67: 7235,
 68: 4950,
 69: 4965,
 70: 4966,
 71: 15055,
 72: 3399,
 73: 1846,
 74: 3968,
 75: 2922,
 76: 5067,
 77: 2923,
 78: 2913,
 79: 5106,
 80: 2895,
 81: 3752,
 82: 3767,
 83: 2585,
 84: 2723,
 85: 2724,
 86: 2725,
 87: 3716,
 88: 3698,
 89: 3699,
 90: 

In [20]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

In [21]:
new_route_dict = {}
for key in new_route_label_dict.keys():
    new_route_dict[key] = route_dict[str(new_route_label_dict[key])]

In [22]:
new_route_dict

{0: {u'desti': 1,
  u'length_of_route': 20.61412691556436,
  u'link_idx_list': [59, 54, 25, 13],
  u'node_list': [22, 16, 17, 6, 4],
  u'origi': 11},
 1: {u'desti': 1,
  u'length_of_route': 21.150250340488512,
  u'link_idx_list': [61, 25, 13],
  u'node_list': [22, 17, 6, 4],
  u'origi': 11},
 2: {u'desti': 1,
  u'length_of_route': 22.13084965216668,
  u'link_idx_list': [59, 33, 21, 13],
  u'node_list': [22, 16, 8, 6, 4],
  u'origi': 11},
 3: {u'desti': 8,
  u'length_of_route': 3.4291045722582214,
  u'link_idx_list': [59],
  u'node_list': [22, 16],
  u'origi': 11},
 4: {u'desti': 8,
  u'length_of_route': 4.778901126114534,
  u'link_idx_list': [59, 54],
  u'node_list': [22, 16, 17],
  u'origi': 11},
 5: {u'desti': 8,
  u'length_of_route': 5.315024551038685,
  u'link_idx_list': [61],
  u'node_list': [22, 17],
  u'origi': 11},
 6: {u'desti': 12,
  u'length_of_route': 4.583763012769106,
  u'link_idx_list': [78],
  u'node_list': [22, 23],
  u'origi': 11},
 7: {u'desti': 12,
  u'length_of_rou

In [23]:
with open('../temp_files/new_route_dict_journal.json', 'w') as json_file:
    json.dump(new_route_dict, json_file)

In [24]:
with open('../temp_files/new_route_dict_journal.json', 'r') as json_file:
    new_route_dict = json.load(json_file)

In [25]:
# number of routes
number_of_routes = len(new_route_dict)

In [26]:
number_of_routes

270

In [27]:
# initialize the route_choice_probability_matrix P
P = np.zeros((number_of_od_pairs, number_of_routes))

np.shape(P)

(90, 270)

In [28]:
for row in range(number_of_od_pairs):
    for column in range(number_of_routes):
        if new_route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        new_route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            P[row, column] = 1

In [29]:
# for column in range(r):
#     assert(sum(P[:, column]) <= 1)

In [30]:
# for row in range(i):
#     assert(sum(P[row, :]) >= 1)

### Use sparse matrix to save memory

In [31]:
P = csr_matrix(P)

In [32]:
P

<90x270 sparse matrix of type '<type 'numpy.float64'>'
	with 270 stored elements in Compressed Sparse Row format>

In [33]:
zdump(P, '../temp_files/OD_pair_route_incidence_journal.pkz')

In [34]:
# link_label_dict = zload('../temp_files/link_label_dict_journal.pkz')
# link_label_dict_ = zload('../temp_files/link_label_dict_journal_.pkz')

In [35]:
# len(link_label_dict)

In [36]:
## number of links
# number_of_links = len(link_label_dict)

In [37]:
# number_of_links

In [38]:
feasible_link_set = set({})
for key in new_route_dict.keys():
    feasible_link_set = feasible_link_set.union(set(new_route_dict[key]['link_idx_list']))

In [39]:
len(feasible_link_set)

122

In [40]:
feasible_link_set

{12,
 13,
 14,
 16,
 17,
 18,
 19,
 21,
 24,
 25,
 32,
 33,
 38,
 39,
 42,
 43,
 54,
 55,
 58,
 59,
 60,
 61,
 70,
 71,
 72,
 73,
 78,
 79,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 94,
 95,
 96,
 97,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 110,
 111,
 112,
 113,
 116,
 117,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 131,
 132,
 133,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 159,
 164,
 165,
 166,
 167,
 168,
 169,
 174,
 175,
 176,
 177,
 182,
 183,
 222,
 223,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 244,
 245,
 250,
 251,
 254,
 255}

In [41]:
feasible_link_list = list(feasible_link_set)

feasible_link_dict = {}
for idx in range(len(feasible_link_list)):
    feasible_link_dict[idx] = feasible_link_list[idx]
    
zdump(feasible_link_dict, '../temp_files/feasible_link_dict_journal.pkz')

In [42]:
feasible_link_dict

{0: 12,
 1: 13,
 2: 14,
 3: 16,
 4: 17,
 5: 18,
 6: 19,
 7: 21,
 8: 24,
 9: 25,
 10: 32,
 11: 33,
 12: 38,
 13: 39,
 14: 42,
 15: 43,
 16: 54,
 17: 55,
 18: 58,
 19: 59,
 20: 60,
 21: 61,
 22: 70,
 23: 71,
 24: 72,
 25: 73,
 26: 78,
 27: 79,
 28: 82,
 29: 83,
 30: 84,
 31: 85,
 32: 86,
 33: 87,
 34: 88,
 35: 89,
 36: 90,
 37: 91,
 38: 94,
 39: 95,
 40: 96,
 41: 97,
 42: 100,
 43: 101,
 44: 102,
 45: 103,
 46: 104,
 47: 105,
 48: 106,
 49: 107,
 50: 110,
 51: 111,
 52: 112,
 53: 113,
 54: 116,
 55: 117,
 56: 118,
 57: 119,
 58: 120,
 59: 122,
 60: 123,
 61: 124,
 62: 125,
 63: 126,
 64: 127,
 65: 128,
 66: 129,
 67: 131,
 68: 132,
 69: 133,
 70: 136,
 71: 137,
 72: 138,
 73: 139,
 74: 140,
 75: 141,
 76: 142,
 77: 143,
 78: 144,
 79: 145,
 80: 146,
 81: 148,
 82: 149,
 83: 150,
 84: 151,
 85: 152,
 86: 153,
 87: 154,
 88: 155,
 89: 159,
 90: 164,
 91: 165,
 92: 166,
 93: 167,
 94: 168,
 95: 169,
 96: 174,
 97: 175,
 98: 176,
 99: 177,
 100: 182,
 101: 183,
 102: 222,
 103: 223,
 104: 23

In [43]:
# initialize the link_route incidence matrix
number_of_links = len(feasible_link_dict)
A = np.zeros((number_of_links, number_of_routes))

# create link_route incidence matrix A
# route index
for i in range(number_of_routes):
    # link index
    for j in range(number_of_links):
        if feasible_link_dict[j] in new_route_dict[str(i)]['link_idx_list']:
            A[j, i] = 1

### Use sparse matrix to save memory

In [44]:
A = csc_matrix(A)        

zdump(A, '../temp_files/link_route_incidence_matrix_journal.pkz')

In [45]:
matrix_rank(A.todense())

96

In [46]:
print(np.size(A, 0))
print(np.size(A, 1))

122
270
